In [1]:
import math
from importlib import reload

import numpy as np
import pandas as pd

In [2]:
import BSM_call_greeks
import BSM_option_valuation

reload(BSM_option_valuation)
reload(BSM_call_greeks)
from BSM_call_greeks import *
from BSM_option_valuation import *

## Continous-Time Delta  
Pg 118-121

In [35]:
s = 100
r = 0
expiry = 30 / 365
vol = 0.157

delta1 = BSM_delta(s, 96, 0, expiry, r, vol)
print("Delta for long $1m of 96 calls: ", delta1)
delta1_val = delta1 * 10**6
print("Continuous Delta : $", delta1_val)

Delta for long $1m of 96 calls:  0.8236717891061237
Continuous Delta : $ 823671.7891061237


In [36]:
delta2 = BSM_delta(s, 104, 0, expiry, r, vol)
print("Delta for short $1m of 96 calls: ", delta2)
delta2_val = delta2 * 10**6
print("Continuous Delta : $", delta2_val)

Delta for short $1m of 96 calls:  0.1979788166354165
Continuous Delta : $ 197978.8166354165


In [37]:
tot_delta = delta1 - delta2
tot_delta_val = delta1_val - delta2_val
print("Total Continuous Delta: $", tot_delta_val)

Total Continuous Delta: $ 625692.9724707073


In [38]:
def get_delta(s, short):
    d1 = BSM_delta(s, 96, 0, expiry, r, vol)
    d2 = BSM_delta(s, 104, 0, expiry, r, vol)
    return short + d2 - d1


def get_pl(s):
    init_l_call_contracts = 10**6 / BSM_call_value(100, 96, 0, expiry, r, vol)
    init_s_call_contracts = 10**6 / BSM_call_value(100, 104, 0, expiry, r, vol)
    shares_short = 626000 / 100

    pl_l = BSM_call_value(s, 96, 0, expiry, r, vol) * init_l_call_contracts - 10**6
    # print(pl_l)

    pl_s = BSM_call_value(s, 104, 0, expiry, r, vol) * init_s_call_contracts - 10**6
    # print(pl_s)

    pl_shares = shares_short * s - 626000
    # print(pl_shares)

    print(pl_l, pl_s, pl_shares)
    print(pl_l - pl_s + pl_shares)


#     init_price_longc = BSM_call_value(100, 96, 0, expiry, r, vol)*10**6
#     init_price_shortc = BSM_call_value(100, 104, 0, expiry, r, vol)*10**6
#     init_price_short_shares = 626000/100

#     new_price_longc = BSM_call_value(s, 96, 0, expiry, r, vol)*init_l_call_contracts
#     new_price_shortc = BSM_call_value(s, 104, 0, expiry, r, vol)*10**6
#     shares_val = init_price_short_shares * s

#     a = new_price_longc - init_price_longc
#     b = init_price_shortc - new_price_shortc
#     c = 626000-shares_val
#     print(a)
#     print(b)
#     print(c)
#     print(a+b+c)

In [39]:
asset_prices = [92.5, 93.5, 95.5] + list(range(97, 108)) + [108.5]
deltas = []

for price in asset_prices:
    # get_pl(price)
    deltas.append(round(get_delta(price, tot_delta) * 1000))

df = pd.DataFrame(
    {
        "Asset Price": asset_prices,
        # 'P/L': pls,
        "Delta 000": deltas,
    }
)
df

,Asset Price,Delta 000
0,92.5,420
1,93.5,349
2,95.5,194
3,97.0,89
4,98.0,38
5,99.0,8
6,100.0,0
7,101.0,16
8,102.0,52
9,103.0,106


In [40]:
# TODO: get P/L working

## Asset Values One Period Ahead  
Pg 129

In [41]:
t = 0.12
mu = 0
S_0 = 100
sigmas = [0.1, 0.157, 0.25, 0.5, 1]
Zs = list(range(-3, 4))

In [42]:
def geo_brown(S_0, mu, sigma, t, Z):
    return S_0 * np.exp((mu - 0.5 * (sigma**2)) * t + sigma * math.sqrt(t) * Z)

In [43]:
geo_brown(S_0, mu, sigmas[0], t, Zs[0])

90.07540361795392

In [44]:
for Z in Zs:
    print("\nZ: ", Z)
    for sigma in sigmas:
        print(geo_brown(S_0, mu, sigma, t, Z))


Z:  -3
90.07540361795392
84.82005902360015
76.8313350076207
58.58946742873772
33.31273317234942

Z:  -2
93.25038187458354
89.56086547102224
83.78174178020322
69.66931831250255
47.103620336418984

Z:  -1
96.53727177995606
94.56664751538025
91.36090444124676
82.84447917251491
66.6036928677811

Z:  0
99.94001799640054
99.85221530928224
99.6257022469171
98.51119396030626
94.17645335842487

Z:  1
103.4627042277226
105.43320678201763
108.63815993168313
117.14064029875276
133.16385301305067

Z:  2
107.10955812013903
111.32613390607801
118.46591318463884
139.29310018444033
188.29135221086594

Z:  3
110.88495633596077
117.54843154963027
129.18271623429325
165.63481051075598
266.24066903442446


## Deltas with Extreme Shifts in Volatility
pg 131

In [69]:
s = 100
r = 0
expiry = 180 / 365
strikes = [110, 90]  # todo: add 90 put
vols = [10, 15] + list(range(20, 90, 10)) + list(range(100, 200, 20))
vols = [x * 0.01 for x in vols]
deltas = []
for strike in strikes:
    d = []
    for vol in vols:
        d.append((round(BSM_delta(s, strike, 0, expiry, r, vol), 2)))
    deltas.append(d)


df = pd.DataFrame(
    {
        "VOL": vols,
        "90 Put Delta": [x - 1 for x in deltas[1]],
        "110 Call Delta": deltas[0],
        "90 Call Delta": deltas[1],
    }
)
df

,VOL,90 Put Delta,110 Call Delta,90 Call Delta
0,0.10,-0.06,0.09,0.94
1,0.15,-0.15,0.20,0.85
2,0.20,-0.21,0.27,0.79
3,0.30,-0.27,0.36,0.73
4,0.40,-0.30,0.42,0.70
5,0.50,-0.32,0.46,0.68
6,0.60,-0.32,0.49,0.68
7,0.70,-0.32,0.52,0.68
8,0.80,-0.32,0.54,0.68
9,1.00,-0.31,0.59,0.69
